# Notebook 1 – Build Example Factors and Structural Betas


In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

from src.factors import build_tail_factor
from src.unified_model import get_structural_betas, factor_returns_to_privates


## Create a synthetic factor dataset (for demo)


In [2]:
idx = pd.date_range('2005-01-31', periods=180, freq='M')
rng = np.random.default_rng(42)

SC = pd.Series(rng.normal(0.008, 0.05, size=len(idx)), index=idx, name='SC')
CS = pd.Series(rng.normal(0.000, 0.02, size=len(idx)), index=idx, name='CS')
INNOV = pd.Series(rng.normal(0.004, 0.04, size=len(idx)), index=idx, name='INNOV')
dVIX = pd.Series(rng.normal(0.0, 2.0, size=len(idx)), index=idx, name='dVIX')
TAIL = build_tail_factor(dVIX)

factors = pd.concat([SC, CS, INNOV, TAIL], axis=1).dropna()
factors.tail()

/var/folders/55/623vgtzj3hn7ffs5kj635y1c0000gn/T/ipykernel_91479/2961191251.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  idx = pd.date_range('2005-01-31', periods=180, freq='M')


,SC,CS,INNOV,TAIL
2019-08-31,0.016629,0.004658,0.029499,0.000000
2019-09-30,0.086955,0.004688,-0.017551,0.000000
2019-10-31,0.016000,0.005407,0.049913,0.637545
2019-11-30,0.002068,-0.017267,-0.091770,0.000000
2019-12-31,0.022291,-0.002951,-0.027463,0.000000


## Structural betas for Buyout and VC


In [3]:
betas_bo = get_structural_betas('buyout')
betas_vc = get_structural_betas('vc')
print('Buyout betas:')
print(betas_bo)
print('\nVC betas:')
print(betas_vc)

Buyout betas:
SC       1.5
CS       0.8
INNOV    0.1
TAIL     1.2
dtype: float64

VC betas:
SC       1.2
CS       0.4
INNOV    1.1
TAIL     2.0
dtype: float64


## Factor-implied returns for Buyout and VC


In [4]:
bo_returns = factor_returns_to_privates(factors, betas_bo, eps_sigma=0.02)
vc_returns = factor_returns_to_privates(factors, betas_vc, eps_sigma=0.03)
df_priv = pd.concat([bo_returns.rename('Buyout'), vc_returns.rename('VC')], axis=1)
df_priv.tail()

,Buyout,VC
2019-08-31,0.046592,0.054984
2019-09-30,0.132170,0.120631
2019-10-31,0.811385,1.359705
2019-11-30,-0.003137,-0.095748
2019-12-31,0.042050,-0.009471
